#### Libraries importeren

In [195]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image

#### Dataset inladen en inzien

In [196]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [197]:
image, label = train_data[0]
print(f"Afbeelding formaat: {image.shape}")

Afbeelding formaat: torch.Size([1, 28, 28])


#### Set seed

In [198]:
np.random.seed(42)
torch.manual_seed(42)

#### Architectuur van CNN bouwen

In [199]:
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Flatten(),
    
    nn.Linear(128 * 3 * 3, 128),
    nn.ReLU(),
    
    nn.Linear(128, 10)
)

#### Check voor GPU
I.v.m. dat het model zwaar kan zijn, check ik eerst ff of het huidige apparaat een GPU heeft. Dit kan het trainen een stuk versneller namelijk.

In [200]:
component = torch.device("cuda" if torch.cuda.is_available()
                         else "cpu")
model = model.to(component)
print(f"Model runt op {component}")

Model runt op cuda


#### Model trainen

In [201]:
optimalisator = optim.Adam(model.parameters(), lr=0.001)
criterium = nn.CrossEntropyLoss()

In [202]:
epochs = 10
for epoch in range(epochs):
    model.train()
    totale_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(component), labels.to(component) #Runt het model op geschikte component
        
        optimalisator.zero_grad()
        output = model(images)
        loss = criterium(output, labels)
        loss.backward()
        optimalisator.step()
        totale_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {totale_loss / len(train_loader)}")

torch.save(model.state_dict(), "Getrainde_modellen/CNN_Model_gewichten.pth")

Epoch 1, Loss: 0.17335313412662187
Epoch 2, Loss: 0.04524879490319731
Epoch 3, Loss: 0.03176438094858988
Epoch 4, Loss: 0.024270170764297435
Epoch 5, Loss: 0.020867435494088753
Epoch 6, Loss: 0.01680532881355655
Epoch 7, Loss: 0.013147783664775535
Epoch 8, Loss: 0.011724266275831702
Epoch 9, Loss: 0.011341556380089128
Epoch 10, Loss: 0.008449799845600202


#### Laad getrainde model indien nodig (voor apparaten zonder cuda bijvoorbeeld)

In [203]:
model.load_state_dict(torch.load("Getrainde_modellen/CNN_Model_gewichten.pth", 
                                 map_location=torch.device(component)))


<All keys matched successfully>

#### Evaluatie van model

In [204]:
model.eval()
with torch.no_grad():
    correct_voorspeld = 0
    totaal_voorspeld = 0
    for images, labels in test_loader:
        images, labels = images.to(component), labels.to(component)
        test_output = model(images)
        _, predicted = torch.max(test_output, 1)
        correct_voorspeld += (predicted == labels).sum().item()
        totaal_voorspeld += labels.size(0)

accuratie = correct_voorspeld / totaal_voorspeld * 100

print("Werkelijke labels:", labels[:16].tolist())  
print("Voorspelde labels:", predicted[:16].tolist()) 
print(f"Accuratie na training: {accuratie:.2f}%")

Werkelijke labels: [8, 0, 0, 3, 2, 2, 9, 7, 2, 7, 9, 0, 4, 7, 1, 4]
Voorspelde labels: [8, 0, 0, 3, 2, 2, 9, 7, 2, 7, 9, 0, 4, 7, 1, 4]
Accuratie na training: 99.12%


#### Nieuwe data inladen en transformeren

In [ ]:
Nummer1 = Image.open("Afbeeldingen/Nummer1.jpg").convert("L")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),  
    transforms.ToTensor(),         
    transforms.Lambda(lambda x: 1.0 - x),
    transforms.Normalize((0.5,), (0.5,)) 
])
Nummer1 = transform(Nummer1)
Nummer1 = Nummer1.unsqueeze(0).to(component)

#### Nieuwe data introduceren in CNN

In [212]:
model.eval()
with torch.no_grad():
    test_output = model(Nummer1)
    zekerheidsgraad = torch.softmax(test_output, dim=1)
    voorspelling = torch.argmax(zekerheidsgraad, 1).item()
    zekerheid = zekerheidsgraad[0][voorspelling].item()
    
    print(f"Het model denkt dat dit cijfer een {voorspelling} is met een zekerheidsgraad van {zekerheid:.4f}!")


Het model denkt dat dit cijfer een 1 is met een zekerheidsgraad van 0.1161!
